In [1]:
import papermill as pm
import os
from prefect.storage import Azure
import prefect
from prefect import task, Flow
import fsspec
from prefect.tasks.jupyter import ExecuteNotebook

/srv/conda/envs/notebook/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
connection_string = os.environ.get("AZURE_STORAGE_CONNECTION_STRING")

In [5]:
storage = Azure("prefect")
image = "carbonplan/cmip6-downscaling-prefect:latest"

In [6]:
out_path = "az://cmip6"

In [4]:
parameters = {'run_id': 'gcm_name',
             'a': 3}

In [5]:
# out = fsspec.get_mapper('az://cmip6downscaling/scratch/papermill_test{}.ipynb'.format(parameters['run_id']),
#                           connection_string=connection_string)
# out = "abs://cmip6downscaling.blob.core.windows.net/cmip6/temp/papermill_test.ipynb"

In [6]:
@task
def run_papermill_task(parameters):
    pm.execute_notebook(
           'papermill_test.ipynb', # can't figure out how to write to azure. instead can write local, convert to html
           'analyses{}.ipynb'.format(parameters['run_id']),
           parameters=parameters
        )

In [7]:
with Flow(name=parameters['run_id']) as flow:
    run_papermill_task(parameters)

In [8]:
flow.run()

[2022-01-10 20:43:00+0000] INFO - prefect.FlowRunner | Beginning Flow run for 'gcm_name'
[2022-01-10 20:43:00+0000] INFO - prefect.TaskRunner | Task 'run_papermill_task': Starting task run...


Executing:   0%|          | 0/6 [00:00<?, ?cell/s]

[2022-01-10 20:43:04+0000] INFO - prefect.TaskRunner | Task 'run_papermill_task': Finished task run for task with final state: 'Success'
[2022-01-10 20:43:04+0000] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded


<Success: "All reference tasks succeeded.">

In [20]:
# this might be what we want to do eventually but it struggles - AttributeError: 'OutStream' object has no attribute 'buffer'

In [27]:
task = ExecuteNotebook('papermill_test.ipynb',
                       parameters=parameters, output_format='html', kernel_name='papermill-tutorial')
output = task.run()